My goal from this project in the LAKAthon was looking at the number of success verses attempts in each project. I wanted to see which shapes were used by everyone that was making an attempt verses the ones that failed. 
```
Puzzle Design a Puzzle           Attempts  1257 / Success  0    :  0.0 % Solution Shapes: []
Shapes Used Total: Counter({1: 33, 4: 23, 5: 18, 2: 11, 3: 6, 6: 5})
Shapes Used Success: Counter({1: 3, 4: 3, 2: 3, 5: 1, 3: 1})
Puzzle Pyramids are Strange      Attempts  1050 / Success  6    :  0.6 % Solution Shapes: [2, 2, 2, 2]
Shapes Used Total: Counter({2: 48})
Shapes Used Success: Counter({2: 31})
Puzzle Scaling Round Objects     Attempts  767  / Success  7    :  0.9 % Solution Shapes: [4, 2]
Shapes Used Total: Counter({4: 13, 5: 13, 6: 1, 1: 1})
Shapes Used Success: Counter({4: 10, 5: 9, 6: 1})
Puzzle Cubes Obscure Spheres     Attempts  724  / Success  6    :  0.8 % Solution Shapes: [6, 4, 4, 1]
Shapes Used Total: Counter({1: 33, 4: 27, 6: 11})
Shapes Used Success: Counter({1: 14, 4: 13, 6: 7})
Puzzle Not Bird                  Attempts  647  / Success  3    :  0.5 % Solution Shapes: [6, 4, 2]
Shapes Used Total: Counter({1: 10, 4: 9, 6: 6, 5: 6, 2: 5, 3: 2})
Shapes Used Success: Counter({4: 5, 5: 4, 2: 3, 6: 2})
Puzzle Bird Fez                  Attempts  635  / Success  8    :  1.3 % Solution Shapes: [6, 5, 4]
Shapes Used Total: Counter({1: 9, 6: 8, 5: 5, 4: 3, 2: 3})
Shapes Used Success: Counter({1: 9, 6: 7, 5: 4, 2: 3, 4: 2})
Puzzle Rotate a Pyramid          Attempts  629  / Success  10   :  1.6 % Solution Shapes: [2]
Shapes Used Total: Counter({2: 20})
Shapes Used Success: Counter({2: 12})
Puzzle 45-Degree Rotations       Attempts  603  / Success  9    :  1.5 % Solution Shapes: [3, 3, 3, 3, 6]
Shapes Used Total: Counter({3: 45, 6: 9})
Shapes Used Success: Counter({3: 45, 6: 9})
Puzzle Angled Silhouette         Attempts  527  / Success  5    :  0.9 % Solution Shapes: [1, 1, 2, 2]
Shapes Used Total: Counter({1: 15, 2: 13, 3: 6})
Shapes Used Success: Counter({1: 15, 2: 13, 3: 6})
Puzzle Square Cross-Sections     Attempts  520  / Success  8    :  1.5 % Solution Shapes: [3, 4, 2]
Shapes Used Total: Counter({3: 11, 2: 10, 4: 8, 6: 6, 1: 2, 5: 1})
Shapes Used Success: Counter({3: 9, 2: 8, 4: 8, 6: 4, 1: 1})
Puzzle Pi Henge                  Attempts  421  / Success  7    :  1.7 % Solution Shapes: [1, 1, 1, 1, 1]
Shapes Used Total: Counter({1: 32, 4: 3})
Shapes Used Success: Counter({1: 32, 4: 3})
Puzzle Stack 2 Ramps             Attempts  401  / Success  8    :  2.0 % Solution Shapes: [3, 3]
Shapes Used Total: Counter({3: 21})
Shapes Used Success: Counter({3: 19})
Puzzle Shape Limits              Attempts  389  / Success  6    :  1.5 % Solution Shapes: [1]
Shapes Used Total: Counter({1: 12})
Shapes Used Success: Counter({1: 9})
Puzzle Match Silhouettes         Attempts  341  / Success  10   :  2.9 % Solution Shapes: [1, 2]
Shapes Used Total: Counter({1: 19, 2: 15})
Shapes Used Success: Counter({1: 12, 2: 12})
Puzzle Max 2 Cubes               Attempts  270  / Success  8    :  3.0 % Solution Shapes: [1, 1]
Shapes Used Total: Counter({1: 20})
Shapes Used Success: Counter({1: 15})
Puzzle Stretch a Ramp            Attempts  229  / Success  9    :  3.9 % Solution Shapes: [3]
Shapes Used Total: Counter({3: 12})
Shapes Used Success: Counter({3: 12})
Puzzle Removing Objects          Attempts  172  / Success  9    :  5.2 % Solution Shapes: [2, 5]
Shapes Used Total: Counter({2: 12, 5: 9, 1: 5})
Shapes Used Success: Counter({2: 10, 5: 9, 1: 4})
Puzzle Separated Cubes           Attempts  144  / Success  10   :  6.9 % Solution Shapes: [1, 1]
Shapes Used Total: Counter({1: 22})
Shapes Used Success: Counter({1: 20})
Puzzle One Cube                  Attempts  99   / Success  10   :  10.1% Solution Shapes: [1]
Shapes Used Total: Counter({1: 10})
Shapes Used Success: Counter({1: 9})
Puzzle Sandbox                   Attempts  10   / Success  0    :  0.0 % Solution Shapes: []
Shapes Used Total: Counter()
Shapes Used Success: Counter()
Puzzle Design a Puzzle 2         Attempts  3    / Success  3    :  100.0% Solution Shapes: []
Shapes Used Total: Counter()
Shapes Used Success: Counter()
Puzzle Design a Puzzle 1         Attempts  1    / Success  1    :  100.0% Solution Shapes: []
Shapes Used Total: Counter()
Shapes Used Success: Counter()
```
Some interesting things I found.

Some of the solutions used shapes that weren’t in the actual solution, so there were many people completing the puzzle using alternate solutions.

Even though many of the puzzles had a low success rate and few puzzles were completed , the total actions taken by those who had a success was more than 60% of the total actions. So it seems like many students gave up.

Some of this could be used to help to inform problem difficulty, a system to provide hints to the user based on all possible solutions. It could also be applied to more factors than just shapes. 

In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json, os, collections, ast

In [2]:
playtest_files = ["2018-11-14_Playtest/anonymized_playtest5_data.csv", "2019-01-07_Playtest/anonymized_playtest6_data.csv", "2019-01-31_Playtest/anonymized_playtest7_data.csv"]

playtests = []

for playtest in playtest_files:
    playtests.append(pd.read_csv(playtest, sep=";", index_col=0))
    
# Set the playtest index we want to focus on
playtest = 1


In [3]:
def get_shapes(puzzle):
    shapes = []
    if puzzle is None:
        return shapes
    for shape_data in puzzle.get("shapeData"):
        shapes.append(shape_data.get('shapeType'))
    return shapes    

In [4]:
puzzles = collections.OrderedDict()

with open(f"{os.path.dirname(playtest_files[playtest])}/StreamingAssets/config.json") as f:
    asset_config = json.load(f)

for puzzle_sets in asset_config.get("puzzleSets"):
    for puzzle_file in puzzle_sets.get("puzzles"):
        with open(f"{os.path.dirname(playtest_files[playtest])}/StreamingAssets/{puzzle_file}.json") as f:
            puzzle_details = json.load(f)
            puzzles[puzzle_details.get("puzzleName")] = puzzle_details

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
#all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])
all_playtest_data = pd.concat([playtests[playtest]])

#all_playtest_data = all_playtest_data.join(all_playtest_data['data'].map(json.loads).apply(pd.Series).add_prefix('data.')).drop(["data"], axis=1)
all_playtest_data = all_playtest_data.join(all_playtest_data['data'].map(json.loads).apply(pd.Series)).drop(["data"], axis=1)

In [6]:
# puzzleDict[user~puzzle_id] = {}
puzzleDict = {}
frames = []

# For each session
for user in all_playtest_data['session_id'].unique():
    # Get the sessions
    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
    activePuzzle = None
    for enum, event in user_events.iterrows():
        #print(('{} - {}').format(event['time'], event['type']))
        # Keep track of all frames we've looked at
        if(event['type'] == 'ws-start_level'):
            activePuzzle = event['task_id']
            frames.clear()

            
        # If they are not playing a puzzle we do not do anything and continue
        if(activePuzzle is None):
            continue

    # Analyze when puzzle is finished or user left
        # Measure time, attempts, completion and actions
        if(event['type'] in ['ws-exit_to_menu', 'ws-disconnect']):
            # This was incomplete so don't reset the active puzzle
            activePuzzle = None
            frames.clear()

        elif (event['type'] in ['ws-puzzle_complete']):
            # print (event['task_id'])
            # This was a correct solution! We need to mark all of these frames as correct
            for frame in frames:
                all_playtest_data.at[frame, 'completed'] = True
            
        else:
            # Set the active puzzle to be this one we're looking at
            all_playtest_data.at[enum, 'task_id'] = activePuzzle
            frames.append(enum)


In [7]:
all_playtest_data['time'] = pd.to_datetime(all_playtest_data['time'])


In [8]:
all_attempts = all_playtest_data.task_id.value_counts()


In [27]:
complete_puzzles = all_playtest_data.loc[all_playtest_data['type'] == 'ws-puzzle_complete']
successful_attempts = complete_puzzles.task_id.value_counts()

In [28]:
all_completed = all_playtest_data.query('completed == True')
print(f"Total {len(all_playtest_data)} Num Completed: {len(all_completed)} {round((len(all_completed) / len(all_playtest_data)) * 100, 1)}%"

SyntaxError: unexpected EOF while parsing (<ipython-input-28-615b03630896>, line 2)

In [22]:
created_shapes = all_playtest_data.query('type == "ws-create_shape"')
success_created_shapes =  created_shapes.query('completed == True')

shapes_used = collections.defaultdict(list)
for shapes, value in created_shapes.iterrows():
    shapes_used[value.get('task_id')].append(int(value.get('shapeType')))

shapes_used_success = collections.defaultdict(list)
for shapes, value in success_created_shapes.iterrows():
    shapes_used_success[value.get('task_id')].append(int(value.get('shapeType')))

In [11]:
for attempt, value in all_attempts.items():
    success_value = successful_attempts.get(attempt, 0)
    shapes = get_shapes(puzzles.get(attempt))
    print (f"Puzzle {attempt: <25} Attempts {value: < 5} / Success {success_value: < 5} : {round(((success_value/value) * 100), 1): < 5}% Solution Shapes: {shapes}")
    times_used = collections.Counter(shapes_used[attempt])
    print (f"Shapes Used Total: {times_used}")
    times_used = collections.Counter(shapes_used_success[attempt])
    print (f"Shapes Used Success: {times_used}")


Puzzle Design a Puzzle           Attempts  1257 / Success  0    :  0.0 % Solution Shapes: []
Shapes Used Total: Counter({1: 33, 4: 23, 5: 18, 2: 11, 3: 6, 6: 5})
Shapes Used Success: Counter({1: 3, 4: 3, 2: 3, 5: 1, 3: 1})
Puzzle Pyramids are Strange      Attempts  1050 / Success  6    :  0.6 % Solution Shapes: [2, 2, 2, 2]
Shapes Used Total: Counter({2: 48})
Shapes Used Success: Counter({2: 31})
Puzzle Scaling Round Objects     Attempts  767  / Success  7    :  0.9 % Solution Shapes: [4, 2]
Shapes Used Total: Counter({4: 13, 5: 13, 6: 1, 1: 1})
Shapes Used Success: Counter({4: 10, 5: 9, 6: 1})
Puzzle Cubes Obscure Spheres     Attempts  724  / Success  6    :  0.8 % Solution Shapes: [6, 4, 4, 1]
Shapes Used Total: Counter({1: 33, 4: 27, 6: 11})
Shapes Used Success: Counter({1: 14, 4: 13, 6: 7})
Puzzle Not Bird                  Attempts  647  / Success  3    :  0.5 % Solution Shapes: [6, 4, 2]
Shapes Used Total: Counter({1: 10, 4: 9, 6: 6, 5: 6, 2: 5, 3: 2})
Shapes Used Success: Counter

In [12]:
# all_playtest_data.type.value_counts()